In [ ]:
!pip install -q av
# !pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers
# !pip install -q flash-attn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 23.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-800bq69h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-800bq69h
  Resolved https://github.com/huggingface/transformers to commit 1d063793318b20654ebb850f48f43e0a247ab7bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10036538 sha256=f8c5cda914b9e198975a1d0923416795f1c0ba9cfe880b05098872f09be348f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-bnpqg19y/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installa

In [ ]:
import av
import numpy as np
from huggingface_hub import hf_hub_download

import torch
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

import torch
import cv2
import numpy as np
from PIL import Image
import requests
import copy
import warnings
from tqdm.notebook import tqdm
import os
import json
from collections import defaultdict

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Load the model in half-precision
model_name = 'llava-hf/llava-onevision-qwen2-0.5b-ov-hf'

model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
    attn_implementation="sdpa",
)

processor = AutoProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/178 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

video_processor/preprocessor_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

In [ ]:
model.config.vocab_size = model.config.text_config.vocab_size

In [ ]:
# For videos we have to feed a "video" type instead of "image"
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "video"},
            {"type": "text", "text": "What is shown in this video?"},
            ],
    },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

In [ ]:
prompt

'<|im_start|>user <video>\nWhat is shown in this video?<|im_end|><|im_start|>assistant\n'

In [ ]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


## for one video

In [ ]:
# Load the video as an np.array, sampling uniformly 8 frames (can sample more for longer videos, up to 32 frames)
video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
# video_path = "/content/5c0a032c998cfe00a955d220_сын унижает и оскорбляет мать как повлиять.mp4"
container = av.open(video_path)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
# indices = np.arange(0, total_frames).astype(int)
video = read_video_pyav(container, indices)

sample_demo_1.mp4:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

In [ ]:
video.shape

(8, 360, 640, 3)

In [ ]:
inputs = processor(videos=list(video), text=prompt, return_tensors="pt").to("cuda:0", torch.float16)

out = model.generate(**inputs,
            do_sample=False,
            temperature=0.5,
            max_new_tokens=4096,
            return_dict_in_generate=True,
            # output_logits=True,
            output_scores=True
                     )

out.keys()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


odict_keys(['sequences', 'scores', 'past_key_values'])

In [ ]:
model.config.vocab_size = model.config.text_config.vocab_size

In [ ]:
transition_scores = model.compute_transition_scores(
    out.sequences, out.scores, normalize_logits=True
)


In [ ]:
transition_scores ,transition_scores.shape

(tensor([[-7.0447e-01, -4.9071e-03, -2.1356e-01, -4.3207e-02, -8.3723e-01,
          -8.1322e-01, -4.6118e-02, -3.3188e-02, -1.3310e-02, -2.9721e-01,
          -9.9138e-01, -1.6689e-06, -7.2667e-04, -1.6497e-03, -3.1691e-01,
          -2.2741e-02, -3.1248e-02, -2.0857e-01, -8.3337e-02]], device='cuda:0'),
 torch.Size([1, 19]))

In [ ]:
input_length = inputs.input_ids.shape[1]
generated_tokens = out.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    score = score.to('cpu')
    print(f"| {tok:5d} | {processor.tokenizer.decode(tok):8s} | {score.numpy():.4f} | {np.exp(score.numpy()):.2%}")

|   785 | The      | -0.7045 | 49.44%
|  2766 |  video   | -0.0049 | 99.51%
|  4933 |  shows   | -0.2136 | 80.77%
|   264 |  a       | -0.0432 | 95.77%
|  1682 |  child   | -0.8372 | 43.29%
| 11699 |  sitting | -0.8132 | 44.34%
|   389 |  on      | -0.0461 | 95.49%
|   264 |  a       | -0.0332 | 96.74%
|  4845 |  bed     | -0.0133 | 98.68%
|    11 | ,        | -0.2972 | 74.29%
|  2922 |  eng     | -0.9914 | 37.11%
|  2128 | ross     | -0.0000 | 100.00%
|   291 | ed       | -0.0007 | 99.93%
|   304 |  in      | -0.0016 | 99.84%
|  5290 |  reading | -0.3169 | 72.84%
|   264 |  a       | -0.0227 | 97.75%
|  2311 |  book    | -0.0312 | 96.92%
|    13 | .        | -0.2086 | 81.17%
| 151645 | <|im_end|> | -0.0833 | 92.00%
